## 恶意挖矿脚本测试文档

### 一、测试环境
Windows11_x64，带有Chrome浏览器和需要的pip库，位于E:\\Sem_8\\Soft_Development\\Tesseract_OCR\\tesseract.exe目录下的OCR引擎。

### 二、测试用例

#### 模块功能函数测试用例
基于这部分的测试主要是通过分类选择测试

![picture 1](images/2a7f2836e70fd5e4af7336eae8dacef8cb057e293abe01260f1c2d41982d6257.png)  

首先是URL判断函数：(Successfully passed)

In [ ]:
def calculate_url_length(url):
    # 计算URL的长度
    return len(url)

print(calculate_url_length(""))
print(calculate_url_length("https://www.baidu.com")) #expected 21

def judge_url_isornothttps(url):
    #判断URL是否是HTTPS协议
    if url.startswith('https'):
        return 1
    return 0

print(judge_url_isornothttps("https://www.baidu.com")) #expected 1
print(judge_url_isornothttps("http://apac.com.tw/")) #expected 0

其次是动态爬虫函数：(Successfully passed)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def GetHtml(url):
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # 无界面模式，可以在后台运行(测试开启)
    chrome_options.add_argument('--ignore-certificate-errors')
    
    # 创建Chrome浏览器实例
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(10)

    # 创建Chrome浏览器实例
    try:   
        driver.get(url)
        html = driver.page_source
    except:
        html = " "
        print("timeout", url)
    finally:
        driver.quit()
        print("success")
        return html
    
html = GetHtml("https://www.google.com/?gws_rd=ssl")
print(html) #expected success and the right page source

html = GetHtml("http://apac.com.tw/")
print(html) #expected success and empty

接着是特征分析中的关键词提取：(Successfully passed)

In [2]:
def KeyWordExtracting(): 
    # 将开源过滤文本下载到测试文档的同一位置
    file_path = "filters.txt"

    with open(file_path, "r") as file:
        text = file.read()

    text = text.splitlines()

    matching_keywords = [] 

    for line in text:
        filtered_letters = list(filter(str.isalnum, line))
        first_letter = filtered_letters[0]
        last_letter = filtered_letters[len(filtered_letters) - 1]
        first_letter_index = line.find(first_letter) if first_letter else -1
        last_letter_index = line.rfind(last_letter) if first_letter else -1

        keyword = line[first_letter_index : last_letter_index + 1]
        matching_keywords.append(keyword)  # 将关键词添加到匹配列表中

    extend_keywords = ["new CoinHive.Anonymous", "new CryptoLoot.Anonymous", "lib/miner.min.js", "new deep.Miner.Anonymous",
                        "new CRLT.Anonymous", "new CoinImp.Anonymous", "new CoinPirate.Anonymous", "ppoi.org/lib/projectpoi.min.js", "new ProjectPoi.Anonymous"]
        
    matching_keywords.extend(extend_keywords)

    return matching_keywords

print(KeyWordExtracting()) #expected the keywords be the same with the filter.txt

['cryptonight.wasm', 'deepMiner.js', 'deepMiner.min.js', 'proxy=wss', 'proxy=ws', 'coinhive.min.js', 'monero-miner.js', 'wasmminer.wasm', 'wasmminer.js', 'cn-asmjs.min.js', 'plugins/aj-cryptominer', 'plugins/ajcryptominer', 'plugins/wp-monero-miner-pro', 'lib/crlt.js', 'pool/direct.js', 'n.2.1.js', 'gridcash.js', 'worker-asmjs.min.js', 'perfekt=wss', 'vbb/me0w.js', 'webmr.js', 'webxmr.js', 'miner.js', 'webmr4.js', 'static/js/tpb.js', 'lib/crypta.js', 'bitrix/js/main/core/core_tasker.js', 'bitrix/js/main/core/core_loader.js', 'load.jsecoin.com', 'coin-hive.com/lib', 'coin-hive.com/proxy', 'coin-hive.com/captcha', 'edgeno.de', 'reasedoper.pw', 'mataharirama.xyz', 'listat.biz', 'lmodr.biz', 'jyhfuqoh.info', 'coinhive.com/lib', 'coinhive.com/proxy', 'coinhive.com/captcha', 'minemytraffic.com/lib', 'crypto-loot.com/lib', 'coin-have.com/c', 'ppoi.org/lib', 'coinerra.com/lib', 'minero.pw/miner.min.js', 'coinblind.com/lib', 'webmine.cz/miner', 'inwemo.com/inwemo.min.js', 'cloudcoins.co/javascr

接着就是前面两个模块结合，并通过ctrl-f判断链接提取是否正确：

In [5]:
import re
def AnalysisHtml(url):
    html = GetHtml(url)
    keywords = KeyWordExtracting()

    pattern = r"\b(?:{})\b".format("|".join(map(re.escape, keywords)))
    html_contains_keywords = re.findall(pattern, html, re.IGNORECASE)

    if html_contains_keywords:
        keywordappeartimes = len(html_contains_keywords)
    else:
        keywordappeartimes = 0
    
    cryptfuncs = ["Cryptonight", "SHA256", "hash", "wasmwrapper", "Web Assembly", "scrypt"]

    pattern = r"\b(?:{})\b".format("|".join(map(re.escape, cryptfuncs)))
    html_contains_cryptfuncs = re.findall(pattern, html, re.IGNORECASE)

    if html_contains_cryptfuncs:
        cryptfunctionappeartimes = len(html_contains_cryptfuncs)
    else:
        cryptfunctionappeartimes = 0

    dynamicfuncs = ["setTimeout", "setInterval"]

    pattern = r"\b(?:{})\b".format("|".join(map(re.escape, dynamicfuncs)))
    html_contains_dynamicfuncs = re.findall(pattern, html, re.IGNORECASE)

    if html_contains_dynamicfuncs:
        dynamicfunctionappeartimes = len(html_contains_cryptfuncs)
    else:
        dynamicfunctionappeartimes = 0

    cpulimits = ["throttle", "throttle:0.9"]
    pattern = r"\b(?:{})\b".format("|".join(map(re.escape, cpulimits)))
    html_contains_cpulimits = re.findall(pattern, html, re.IGNORECASE)

    if html_contains_cpulimits:
        ifcpulimit = 1
    else:
        ifcpulimit = 0

    return keywordappeartimes, cryptfunctionappeartimes, dynamicfunctionappeartimes, ifcpulimit

print(AnalysisHtml("https://www.wikipedia.org/")) #expected success 0000
print(AnalysisHtml("http://digiflake.com/")) #expected success 2000

success
(0, 0, 0, 0)
success
(2, 0, 0, 0)


爬虫登录publicwww.com并爬取相应URL，并联动上述模块一并检测。（Successfully passed）

In [8]:
from selenium.webdriver.common.keys import Keys
from urllib.parse import quote
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

url_list = []  
chrome_options = Options()
# chrome_options.add_argument("--headless")  

driver = webdriver.Chrome(options=chrome_options)

# 打开publicwww.com登录页面
driver.get('https://publicwww.com/profile/login.html')

driver.implicitly_wait(5) 

username_input = driver.find_element(By.XPATH, '//*[@id="input01"]')
password_input = driver.find_element(By.XPATH, '//*[@id="input02"]')
username_input.send_keys('yuntian1215@sjtu.edu.cn') 
password_input.send_keys('2da515c7') 

# 提交表单，完成登录
password_input.send_keys(Keys.RETURN)
driver.implicitly_wait(5) 

def CreateSearchUrl(keyword):
    # 构造搜索URL
    quoted_keyword = '"' + keyword + '"'
    encoded_keyword = quote(quoted_keyword, safe='') 
    search_url = f'https://publicwww.com/websites/{encoded_keyword}'
    return search_url

def GetURL(search_url):
    search_url = str(search_url)
    driver.get(search_url)
    driver.implicitly_wait(10) 

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 提取搜索结果的URL
    link_elements = soup.select('table tbody tr td a')
    for link_element in link_elements:
        url = link_element.get('href')
        if url.startswith('http'):
            url_list.append(url)

# 获取num数量的URL
def GetBadUrl(num):
    keywords = KeyWordExtracting()

    for keyword in keywords:
        search_url = CreateSearchUrl(keyword)
        print(search_url)
        
        GetURL(search_url)
    
        if len(url_list) > num:
            break

    for url in url_list:
        print(url)
   
GetBadUrl(10)
print(url_list)
driver.quit()

https://publicwww.com/websites/%22cryptonight.wasm%22
https://publicwww.com/websites/%22deepMiner.js%22
https://publicwww.com/websites/%22deepMiner.min.js%22
https://publicwww.com/websites/%22proxy%3Dwss%22
https://publicwww.com/websites/%22proxy%3Dws%22
https://publicwww.com/websites/%22coinhive.min.js%22
http://apac.com.tw/
http://thepiratebay.cd/
https://flirt.zone/
https://my-sexy-place.com/
https://threesixty360.wordpress.com/
https://fun-chat.com/
https://sexymaus.com/
https://bdsm-chat.info/
http://proxy.ws/
https://coinrotation.com/
https://coins4days.com/
http://tublogdearquitectura.com/
http://patrickdamiaens.be/
http://www.archiepanjabi.com/
http://youssef-rabbaoui.de/
http://fonts4web.ru/
http://emoticonfun.org/
http://howtolamp.com/
https://peeseaw.blogspot.com/
http://psystems.gr/
http://senior-formation.com/
https://slovenskyviral.com/
http://findgiveaways.blogspot.com/
http://mobi-skidka.com/
['http://apac.com.tw/', 'http://thepiratebay.cd/', 'https://flirt.zone/', 'htt

OCR（图像识别文字功能）（Successfully passed）

In [9]:
import pytesseract
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = 'E:\\Sem_8\\Soft_Development\\Tesseract_OCR\\tesseract.exe'
# 同一路径下命名为test的图片
image = Image.open('test.png')
text = pytesseract.image_to_string(image, lang='chi_sim')

print(text)

colnhive'com/ 册 Dicolnnive Imlin.1sInew CoinHIVe AnonyIm ous
Load jsecolin comjload

(Cryptodoot com|cryptolootpre )Mib/minermin jsnew CryptoLoot Anonym ous
new deep MinerAnonymous|deepMinerjs

Webmine prolliblnew CRLT Anonymous

wwwcoinimp comljscriptsmnew CoinImp Anonymous

wwwceinpirate cflnew CoinPirate Anonymous

ppoiorghib/projectpolmin jsInew ProjectPol .Anonymous


